<p>
  <a href="https://colab.research.google.com/github/neo4j-partners/hands-on-lab-neo4j-and-vertex-ai/blob/main/Lab%205%20-%20Graph%20Data%20Science/embedding.ipynb" target="_blank">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
  </a>
</p>

First off, you'll also need to install a few packages.

In [1]:
!pip install --quiet --upgrade neo4j
!pip install --quiet google-cloud-storage

     |████████████████████████████████| 89 kB 2.7 MB/s 


You'll need to enter the credentials from your Neo4j instance below.

The default DB_NAME is always neo4j.

In [10]:
DB_URL = "neo4j://ec2-44-202-197-32.compute-1.amazonaws.com:7687"
DB_USER = "neo4j"
DB_PASS = "foo123"
DB_NAME = "neo4j"

In [11]:
import pandas as pd
from neo4j import GraphDatabase

driver = GraphDatabase.driver(DB_URL, auth=(DB_USER, DB_PASS))

First we're going to create an in memory graph represtation of the data in Neo4j Graph Data Science (GDS).

In [18]:
with driver.session(database=DB_NAME) as session:
  result = session.read_transaction(
    lambda tx: tx.run(
      """
CALL gds.graph.create(
  'mygraph',                                      
  {
    Company: {},
    Manager: {}
  },
  {
    Owns: {
      properties: {
        shares: {property:'shares', defaultValue:0},
        value: {property:'value', defaultValue: 0}
      }
    }
   }
)
YIELD
  graphName AS graph,
  relationshipProjection AS readProjection,
  nodeCount AS nodes,
  relationshipCount AS rels      """
    ).data()
  )
df = pd.DataFrame(result)
display(df)

,graph,readProjection,nodes,rels
0,mygraph,"{'Owns': {'orientation': 'NATURAL', 'aggregati...",11248,190300


Note, if you get an error saying the graph already exists, that's probably because you ran this code before. You can destroy it using this command:

In [17]:
with driver.session(database=DB_NAME) as session:
  result = session.read_transaction(
    lambda tx: tx.run(
      """
      CALL gds.graph.drop('mygraph')
      """
    ).data()
  )

Now we can generate an embedding from that graph. This is a new feature we can use in our predictions. We're using FastRP, which is a more full featured and higher performance of Node2Vec. You can learn more about that [here](https://neo4j.com/docs/graph-data-science/current/algorithms/fastrp/).

In [19]:
with driver.session(database=DB_NAME) as session:
  result = session.read_transaction(
    lambda tx: tx.run(
      """
        CALL gds.fastRP.mutate('mygraph',{
        iterationWeights: [0.0, 1.00, 1.00, 0.80, 0.60],
        nodeSelfInfluence: 0.15,
        embeddingDimension: 5,
        randomSeed: 1, 
        mutateProperty:'embedding'
        })
      """
    ).data()
  )
df = pd.DataFrame(result)
display(df)

,nodePropertiesWritten,mutateMillis,nodeCount,createMillis,computeMillis,configuration
0,11248,1,11248,1,545,"{'nodeSelfInfluence': 0.15, 'relationshipWeigh..."


In [20]:
with driver.session(database=DB_NAME) as session:
  result = session.read_transaction(
    lambda tx: tx.run(
      """
        CALL gds.graph.streamNodeProperties
        ('mygraph', ['embedding'])
        YIELD nodeId, nodeProperty, propertyValue
        RETURN nodeId, nodeProperty, propertyValue
      """
    ).data()
  )
df = pd.DataFrame(result)
df.head()

,nodeId,nodeProperty,propertyValue
0,0,embedding,"[0.0, -0.1060660257935524, 0.0, 0.0, -0.106066..."
1,1,embedding,"[0.0, 0.0, 0.0, 0.0, 0.0]"
2,2,embedding,"[0.0, 0.0, 0.0, 0.0, 0.0]"
3,3,embedding,"[0.0, 0.0, 0.0, -0.15000000596046448, 0.0]"
4,4,embedding,"[0.0, 0.0, 0.15000000596046448, 0.0, 0.0]"


Now let's grab the relationships.

In [21]:
with driver.session(database=DB_NAME) as session:
  result = session.read_transaction(
    lambda tx: tx.run(
      """
        CALL gds.graph.streamRelationshipProperties
        ('mygraph', ['shares', 'value'])
        YIELD sourceNodeId, targetNodeId, relationshipType, propertyValue
        RETURN sourceNodeId, targetNodeId, relationshipType, propertyValue
      """
    ).data()
  )
df2 = pd.DataFrame(result)
df2.head()

,sourceNodeId,targetNodeId,relationshipType,propertyValue
0,0,3906,Owns,270.0
1,0,3907,Owns,195.0
2,0,3908,Owns,4939.0
3,0,3909,Owns,1557.0
4,0,3910,Owns,837.0


Now we need to take that dataframe and shape it into something that better represents our classification problem.

In [22]:
x = df.pivot(index="nodeId", columns="nodeProperty", values="propertyValue")
x = x.reset_index()
x.columns.name = None
x.head()

,nodeId,embedding
0,0,"[0.0, -0.1060660257935524, 0.0, 0.0, -0.106066..."
1,1,"[0.0, 0.0, 0.0, 0.0, 0.0]"
2,2,"[0.0, 0.0, 0.0, 0.0, 0.0]"
3,3,"[0.0, 0.0, 0.0, -0.15000000596046448, 0.0]"
4,4,"[0.0, 0.0, 0.15000000596046448, 0.0, 0.0]"


Note that the embedding row is an array. To make this dataset more consumable, we should flatten that out into multiple individual features: embedding_0, embedding_1, ... embedding_n.

In [23]:
FEATURES_FILENAME = "features.csv"

embeddings = pd.DataFrame(x["embedding"].values.tolist()).add_prefix("embedding_")
merged = x.drop(columns=["embedding"]).merge(embeddings, left_index=True, right_index=True)
merged

#features_df = merged.drop(columns=["is_fraudster", "num_transactions", "total_transaction_amnt"])
#train_df = merged.drop(columns=["nodeId"])
#features_df.to_csv(FEATURES_FILENAME, index=False)

,nodeId,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4
0,0,0.000000,-0.106066,0.000000,0.000000,-0.106066
1,1,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,0.000000,0.000000,0.000000,-0.150000,0.000000
4,4,0.000000,0.000000,0.150000,0.000000,0.000000
...,...,...,...,...,...,...
11243,11243,0.000000,0.000000,-0.106066,-0.106066,0.000000
11244,11244,-0.075000,0.075000,0.000000,0.075000,-0.075000
11245,11245,0.000000,0.000000,0.000000,0.000000,0.000000
11246,11246,0.000000,0.000000,0.000000,0.000000,0.000000
